# PPT Generator

In [7]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:

from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [3]:
api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

In [4]:
import google.generativeai as genai

In [5]:
genai.configure(api_key=google_api_key)

In [6]:
for m in genai.list_models():
    print(m.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-pro
models/gemini-pro-vision
models/embedding-001
models/aqa


In [7]:
model = genai.GenerativeModel('gemini-pro')

In [11]:
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [9]:
response = model.generate_content("Give me a Quote from Virat Kohli")

In [15]:
response.text

'"Tough times don\'t last. Tough people do." - Virat Kohli'